In [13]:
import ee
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import time
from datetime import datetime

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project='ee-ahtishamali')

startDate = '2024-05-01'
endDate = '2024-05-15'

# Load your GeoDataFrame from CSV
gdf = pd.read_csv('V3_dataset_with_no_crop_points (1).csv')

# Export Function
def export_multiband_image(image, filename_prefix, folder_name, scale, region):
    task = ee.batch.Export.image.toDrive(
        image=image.float(),
        description=filename_prefix,
        folder=folder_name,
        fileNamePrefix=filename_prefix,
        scale=scale,
        region=region,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()

    # Progress monitoring (optional)
    while task.active():
        print('Exporting in progress...')
        time.sleep(20)

    if task.status()['state'] == 'COMPLETED':
        print(f'Export completed successfully: {filename_prefix}')
    else:
        print(f'Export failed: {task.status()}')

# Number of images to process
NumberOfImager = 0

i = 0
for i in range(len(gdf)):

    # Get data for the first row
    row_data = gdf.iloc[i]
    polygon_geom = row_data['polygon_30x30m']
    crop_type = row_data['crop_type']
    Tag_ID = row_data['Tag_ID']

    # if crop_type == "Orchard Orange" or crop_type == "Orchard Guava":
    if crop_type == "Wheat" :
        if NumberOfImager == 100:
            print("1000 images reached")
            break
        NumberOfImager += 1
        print(f"image no = {NumberOfImager}")
        # Convert WKT string to shapely geometry
        polygon = loads(polygon_geom)

        # Convert shapely geometry to GeoJSON dictionary
        polygon_geojson = mapping(polygon)

        print(f"Polygon  {crop_type} = {polygon_geojson}")

        aoi = ee.Geometry(polygon_geojson)

        # Export to Google Drive
        now = datetime.now()
        date_time_str = now.strftime("%Y%m%d_%H%M%S")
        file_name_prefix = f"data_{startDate}_{endDate}_4by4_{crop_type}_{Tag_ID}_{date_time_str}_{NumberOfImager}"
        folder_name = 'EarthEngineImages'  # Folder in Google Drive to save images

        # Filter and Load Sentinel-2 Image Collection
        sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(startDate, endDate) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

        # Select Bands and Resample to 10m
        s2_band_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
        bands_to_resample = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12']

        # Function to resample selected bands to 10m resolution
        def resample_to_10m(image):
            resampled_bands = image.select(bands_to_resample).resample('bilinear')
            original_bands = image.select([band for band in s2_band_names if band not in bands_to_resample])
            return original_bands.addBands(resampled_bands)

        # Apply the resampling function to each image in the collection
        sentinel2_resampled = sentinel2.map(resample_to_10m)

        # Function to create a single image with all selected bands from an ImageCollection
        def create_multiband_image(collection, bands):
            median_image = collection.select(bands).median()
            ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
            evi = median_image.expression(
                '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                {
                    'NIR': median_image.select('B8'),
                    'RED': median_image.select('B4'),
                    'BLUE': median_image.select('B2')
                }
            ).rename('EVI')
            rendvi = median_image.normalizedDifference(['B8A', 'B5']).rename('RENDVI')
            return median_image.addBands([ndvi, evi, rendvi]).float()

        # Create the multiband image with all bands at 10m resolution and added indices
        multiband_image = create_multiband_image(sentinel2_resampled, s2_band_names)

        # Export the multiband image
        export_multiband_image(multiband_image, file_name_prefix, folder_name, 10, aoi)

        print(f"Exporting image with filename prefix: {file_name_prefix}")


image no = 1
Polygon  Wheat = {'type': 'Polygon', 'coordinates': (((74.18344029667625, 31.56237778332375), (74.18344029667625, 31.56264727667625), (74.18317080332375, 31.56264727667625), (74.18317080332375, 31.56237778332375), (74.18344029667625, 31.56237778332375)),)}
Exporting in progress...
Export completed successfully: data_2024-05-01_2024-05-15_4by4_Wheat_53742_20240808_115109_1
Exporting image with filename prefix: data_2024-05-01_2024-05-15_4by4_Wheat_53742_20240808_115109_1
image no = 2
Polygon  Wheat = {'type': 'Polygon', 'coordinates': (((73.80190766667626, 31.734514943323752), (73.80190766667626, 31.73478443667625), (73.80163817332375, 31.73478443667625), (73.80163817332375, 31.734514943323752), (73.80190766667626, 31.734514943323752)),)}
Exporting in progress...
Export completed successfully: data_2024-05-01_2024-05-15_4by4_Wheat_53741_20240808_115130_2
Exporting image with filename prefix: data_2024-05-01_2024-05-15_4by4_Wheat_53741_20240808_115130_2
image no = 3
Polygon 